In [1]:
import pandas as pd
import os

## Fill in header of the data sheet

In [2]:
fh = open("DataSheetHeader.csv")
datasheet_header = fh.read().rstrip()
fh.close()

In [3]:
datasheet_header = datasheet_header.replace("INVESTIGATOR", "Rogier Thurlings")
datasheet_header = datasheet_header.replace("RUNNAME", "runXXX-20201021-basta")
datasheet_header = datasheet_header.replace("DATE", "01-04-2021")
print(datasheet_header)

[Header],,,,,,,,,,,,,,,
IEMFileVersion,4,,,,,,,,,,,,,,
Investigator Name,Rogier Thurlings,,,,,,,,,,,,,,
Experiment Name,runXXX-20201021-basta,,,,,,,,,,,,,,
Date,01-04-2021,,,,,,,,,,,,,,
Workflow,GenerateFASTQ,,,,,,,,,,,,,,
Application,FASTQ Only,,,,,,,,,,,,,,
Assay,Nextera XT v2,,,,,,,,,,,,,,
Description,RNA,,,,,,,,,,,,,,
Chemistry,Amplicon,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Reads],,,,,,,,,,,,,,,
301,,,,,,,,,,,,,,,
301,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Settings],,,,,,,,,,,,,,,
ReverseComplement,0,,,,,,,,,,,,,,
Adapter,CTGTCTCTTATACACATCT,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Data],,,,,,,,,,,,,,,
Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Run,Nr,Sample_Project,Species,Chain,Patient,Sample,UMI


## Read file names from data directory on WebDav server

In [4]:
runs = list()
filenames = list()
fh = open("SAMPLES-basta")
for line in fh:
    line = line.rstrip()
    line = line.split("/")
    runs.append(line[4])
    filenames.append(line[-1])
fh.close()

In [5]:
df_files = pd.DataFrame({'Run': runs, 'Filename': filenames}) 
df_files.head()

,Run,Filename
0,run31-20180921-miseq,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...
1,run31-20180921-miseq,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...
2,run31-20180921-miseq,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...
3,run31-20180921-miseq,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...
4,run31-20180921-miseq,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...


In [6]:
f = lambda x: x.split("_")[1]
df_files["Nr"] = [x for x in map(f, df_files["Filename"])]

In [7]:
df_files.head()

,Run,Filename,Nr
0,run31-20180921-miseq,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...,S1
1,run31-20180921-miseq,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...,S2
2,run31-20180921-miseq,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...,S49
3,run31-20180921-miseq,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...,S3
4,run31-20180921-miseq,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...,S4


In [8]:
# Sample_ID,Sample_Name,Sample
f = lambda x: x.split("_")[0]
df_files["Sample_Name"] = [x for x in map(f, df_files["Filename"])]
df_files["Sample_ID"] = df_files["Sample_Name"]
df_files["Sample"] = df_files["Sample_Name"]
df_files.head()

,Run,Filename,Nr,Sample_Name,Sample_ID,Sample
0,run31-20180921-miseq,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...,S1,B005-SG-B,B005-SG-B,B005-SG-B
1,run31-20180921-miseq,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...,S2,B007-Lnpre-B,B007-Lnpre-B,B007-Lnpre-B
2,run31-20180921-miseq,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...,S49,BASTA-1467-B,BASTA-1467-B,BASTA-1467-B
3,run31-20180921-miseq,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...,S3,B007-Lnpost-B,B007-Lnpost-B,B007-Lnpost-B
4,run31-20180921-miseq,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...,S4,B007-BMDC-B,B007-BMDC-B,B007-BMDC-B


In [9]:
# Sample_Project
df_files["Sample_Project"] = "BASTA"
df_files.head()

,Run,Filename,Nr,Sample_Name,Sample_ID,Sample,Sample_Project
0,run31-20180921-miseq,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...,S1,B005-SG-B,B005-SG-B,B005-SG-B,BASTA
1,run31-20180921-miseq,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...,S2,B007-Lnpre-B,B007-Lnpre-B,B007-Lnpre-B,BASTA
2,run31-20180921-miseq,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...,S49,BASTA-1467-B,BASTA-1467-B,BASTA-1467-B,BASTA
3,run31-20180921-miseq,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...,S3,B007-Lnpost-B,B007-Lnpost-B,B007-Lnpost-B,BASTA
4,run31-20180921-miseq,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...,S4,B007-BMDC-B,B007-BMDC-B,B007-BMDC-B,BASTA


In [10]:
# Species,Patient,UMI
df_files["Species"] = "human"
df_files["Patient"] = "Unknown"
df_files["UMI"] = "yes"

In [11]:
# Chain
df_files["Chain"] = "BCRh"
df_files

,Run,Filename,Nr,Sample_Name,Sample_ID,Sample,Sample_Project,Species,Patient,UMI,Chain
0,run31-20180921-miseq,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...,S1,B005-SG-B,B005-SG-B,B005-SG-B,BASTA,human,Unknown,yes,BCRh
1,run31-20180921-miseq,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...,S2,B007-Lnpre-B,B007-Lnpre-B,B007-Lnpre-B,BASTA,human,Unknown,yes,BCRh
2,run31-20180921-miseq,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...,S49,BASTA-1467-B,BASTA-1467-B,BASTA-1467-B,BASTA,human,Unknown,yes,BCRh
3,run31-20180921-miseq,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...,S3,B007-Lnpost-B,B007-Lnpost-B,B007-Lnpost-B,BASTA,human,Unknown,yes,BCRh
4,run31-20180921-miseq,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...,S4,B007-BMDC-B,B007-BMDC-B,B007-BMDC-B,BASTA,human,Unknown,yes,BCRh
5,run31-20180921-miseq,B007-PBBMDC-1-B_S5_L001.assembled-ATCGATCG-IGH...,S5,B007-PBBMDC-1-B,B007-PBBMDC-1-B,B007-PBBMDC-1-B,BASTA,human,Unknown,yes,BCRh
6,run31-20180921-miseq,B007-PBBMDC-2-B_S6_L001.assembled-ATGCATGC-IGH...,S6,B007-PBBMDC-2-B,B007-PBBMDC-2-B,B007-PBBMDC-2-B,BASTA,human,Unknown,yes,BCRh
7,run31-20180921-miseq,B009-clone2-B_S56_L001.assembled-ACGTACGT-IGH_...,S56,B009-clone2-B,B009-clone2-B,B009-clone2-B,BASTA,human,Unknown,yes,BCRh
8,run31-20180921-miseq,B009-clone6-B_S57_L001.assembled-ACTGACTG-IGH_...,S57,B009-clone6-B,B009-clone6-B,B009-clone6-B,BASTA,human,Unknown,yes,BCRh
9,run31-20180921-miseq,B009-clone10-B_S58_L001.assembled-AGCTAGCT-IGH...,S58,B009-clone10-B,B009-clone10-B,B009-clone10-B,BASTA,human,Unknown,yes,BCRh


In [12]:
# Empty columns for: Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2
df_files["Sample_Plate"] = ""
df_files["Sample_Well"] = ""
df_files["I7_Index_ID"] = ""
df_files["index"] = ""
df_files["I5_Index_ID"] = ""
df_files["index2"] = ""

In [13]:
# Change order of the columns
cols = ["Sample_ID","Sample_Name","Sample_Plate","Sample_Well","I7_Index_ID","index","I5_Index_ID","index2","Run","Nr","Sample_Project","Species","Chain","Patient","Sample","UMI","Filename"]
df_files = df_files[cols]
df_files

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Run,Nr,Sample_Project,Species,Chain,Patient,Sample,UMI,Filename
0,B005-SG-B,B005-SG-B,,,,,,,run31-20180921-miseq,S1,BASTA,human,BCRh,Unknown,B005-SG-B,yes,B005-SG-B_S1_L001.assembled-ACGTACGT-IGH_HUMAN...
1,B007-Lnpre-B,B007-Lnpre-B,,,,,,,run31-20180921-miseq,S2,BASTA,human,BCRh,Unknown,B007-Lnpre-B,yes,B007-Lnpre-B_S2_L001.assembled-ACTGACTG-IGH_HU...
2,BASTA-1467-B,BASTA-1467-B,,,,,,,run31-20180921-miseq,S49,BASTA,human,BCRh,Unknown,BASTA-1467-B,yes,BASTA-1467-B_S49_L001.assembled-ACTGACTG-IGH_H...
3,B007-Lnpost-B,B007-Lnpost-B,,,,,,,run31-20180921-miseq,S3,BASTA,human,BCRh,Unknown,B007-Lnpost-B,yes,B007-Lnpost-B_S3_L001.assembled-AGCTAGCT-IGH_H...
4,B007-BMDC-B,B007-BMDC-B,,,,,,,run31-20180921-miseq,S4,BASTA,human,BCRh,Unknown,B007-BMDC-B,yes,B007-BMDC-B_S4_L001.assembled-AGTCAGTC-IGH_HUM...
5,B007-PBBMDC-1-B,B007-PBBMDC-1-B,,,,,,,run31-20180921-miseq,S5,BASTA,human,BCRh,Unknown,B007-PBBMDC-1-B,yes,B007-PBBMDC-1-B_S5_L001.assembled-ATCGATCG-IGH...
6,B007-PBBMDC-2-B,B007-PBBMDC-2-B,,,,,,,run31-20180921-miseq,S6,BASTA,human,BCRh,Unknown,B007-PBBMDC-2-B,yes,B007-PBBMDC-2-B_S6_L001.assembled-ATGCATGC-IGH...
7,B009-clone2-B,B009-clone2-B,,,,,,,run31-20180921-miseq,S56,BASTA,human,BCRh,Unknown,B009-clone2-B,yes,B009-clone2-B_S56_L001.assembled-ACGTACGT-IGH_...
8,B009-clone6-B,B009-clone6-B,,,,,,,run31-20180921-miseq,S57,BASTA,human,BCRh,Unknown,B009-clone6-B,yes,B009-clone6-B_S57_L001.assembled-ACTGACTG-IGH_...
9,B009-clone10-B,B009-clone10-B,,,,,,,run31-20180921-miseq,S58,BASTA,human,BCRh,Unknown,B009-clone10-B,yes,B009-clone10-B_S58_L001.assembled-AGCTAGCT-IGH...


# Glue together the header and the values

In [14]:
df_files.to_csv("tmp.csv", header=None, index=None)

In [15]:
fh = open("tmp.csv")
values = fh.read().rstrip()
fh.close()

In [16]:
fhOut = open("20210401-DataSheet-BASTA.csv", "w")
print(datasheet_header + ",filename", file=fhOut)
print(values, file=fhOut)
fhOut.close()
print("Wrote 20210401-DataSheet-BASTA.csv to disk")

Wrote 20210401-DataSheet-BASTA.csv to disk
